In [88]:
from langchain_community.document_loaders import PyPDFLoader
from langchain_community.document_loaders import PyPDFDirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import FAISS


from langchain_community.embeddings import HuggingFaceBgeEmbeddings
from langchain.prompts import PromptTemplate

from langchain.chains import RetrievalQA
from dotenv import load_dotenv


import getpass
import os


In [89]:
load_dotenv()

True

In [90]:
os.environ["HUGGINGFACEHUB_API_TOKEN"] = getpass.getpass()


In [92]:
loader = PyPDFDirectoryLoader("./data")
documets = loader.load()

In [93]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size = 1000,chunk_overlap=200)
final_documents = text_splitter.split_documents(documets)

In [94]:
final_documents[0]

Document(metadata={'source': 'data\\The Journey of Asutosh Sidhya.pdf', 'page': 0}, page_content='The Journey of Asutosh Sidhya: A Story of Passion and Innovation  \nBorn on a cool February morning in 1998, in the vibrant city of Bhubaneswar, Odisha, Asutosh Sidhya \nwas a curious child with a natural inclination towards understanding how things worked. The son of a \ngovernment employee and a school teacher, Asutosh grew up in a home where education was highly \nvalued, and curiosity was encouraged. From a young age, he was captivated by the world of technology, \noften spending hours tinkering with old gadgets and taking apart devices to see what made them tick.  \nAsutosh’s fascination with technology only grew stronger as he advanced through school, excelling in \nscience and mathematics. His determination and hard work paid off when he secured admission to the \nIndian Institute of Technology (IIT) Kharagpur, one of t he most prestigious engineering institutions in \nIndia. At IIT

In [95]:
len(final_documents)

5

In [96]:
hugginface_embedding = HuggingFaceBgeEmbeddings(
    model_name = "sentence-transformers/all-MiniLM-L6-v2",
    model_kwargs={"device":"cpu"},
    encode_kwargs={"normalize_embeddings":True}
)

d:\Download\anaconda_install\envs\myenv\lib\site-packages\transformers\tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [81]:
# import numpy as np
# print(np.array(hugginface_embedding.embed_query(final_documents[0].page_content)))
# print(np.array(hugginface_embedding.embed_query(final_documents[0].page_content)).shape)

In [97]:
## VectorStore Creation
vectorstore=FAISS.from_documents(final_documents[:120],hugginface_embedding)

In [105]:
# query="Asutosh siddhya"
# relevant_docments=vectorstore.similarity_search(query)

# print(relevant_docments[0].page_content)

In [106]:
retriever=vectorstore.as_retriever(search_type="similarity",search_kwargs={"k":3})
print(retriever)

tags=['FAISS', 'HuggingFaceBgeEmbeddings'] vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x000001EBCA3074C0> search_kwargs={'k': 3}


In [107]:
from langchain_community.llms import HuggingFaceHub

hf=HuggingFaceHub(
    repo_id="google/gemma-7b",
    model_kwargs={"temperature":0.1,"max_length":100}

)


In [108]:
prompt_template="""
Use the following piece of context to answer the question asked.
Please try to provide the answer only based on the context

{context}
Question:{question}

Helpful Answers:
 """

In [109]:
prompt=PromptTemplate(template=prompt_template,input_variables=["context","question"])

In [110]:
retrievalQA=RetrievalQA.from_chain_type(
    llm=hf,
    chain_type="stuff",
    retriever=retriever,
    return_source_documents=True,
    chain_type_kwargs={"prompt":prompt}
)

In [111]:
query="""Tell me about asutosh sidhya's eaducational life"""

In [87]:
# Call the QA chain with our query.
result = retrievalQA.invoke({"query": query})
print(result['result'])

ConnectionError: (ProtocolError('Connection aborted.', RemoteDisconnected('Remote end closed connection without response')), '(Request ID: 0d25b0ec-336c-405d-8478-51d19168ab2e)')